<a href="https://colab.research.google.com/github/williamtbarker/ML4Molecules/blob/main/exercise_4_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Task

For this task you will use the QM9 dataset with HOMO as the target value. Perform the following -
1. split the dataset with RandomSplitter, ScaffoldSplitter and MolecularWeightSplitterfrom deepchem. You can limit the split to train-test split with 80:20 split. You can use any featurizer.
2. for each of the above splits, compare the score on the the test dataset with `MLPPredictor` model for 30 epochs. Does the splitting method affect the model performance?
3. for the random split dataset split, change the `hidden_feats` parameter during model creating and train the model again to 30 epochs. Does the R2 score improve with increasing the `hidden_feats`?
4. Use the `CircularFingerprint` featurizer and split the dataset randomly. First set the size of the fingerprint to 100. Train the `MLPPredictor` model with 30 epochs. Then change the size of the fingerprints and repeat the training. You can choose the `hidden_feats` our your choice but keep it constant over different fingerprint length. Do the length of fingerprint play a role?

In [1]:
!pip install deepchem dgl dgllife rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
jax 0.4.23 requires scipy>=1.9, but you have

Remember to install all the packages

In [4]:
import pandas as pd
import deepchem as dc
from rdkit import Chem
from dgllife.model.model_zoo.mlp_predictor import MLPPredictor

# Load and sample the dataset
df = pd.read_csv("https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
dataset = df[["smiles", "homo"]].sample(frac=0.05)

# Ensure that the 'smiles' column is a string and check for null values
dataset['smiles'] = dataset['smiles'].astype(str)
assert dataset['smiles'].apply(lambda x: isinstance(x, str)).all()
assert not dataset['smiles'].isnull().any()

# Create the featurizer object
featurizer = dc.feat.CircularFingerprint(size=100, radius=2)

# Featurize the dataset
features = [list(featurizer.featurize(smiles)[0]) for smiles in dataset['smiles']]
targets = dataset['homo'].values.reshape(-1, 1)

# Create a DeepChem dataset
dc_dataset = dc.data.NumpyDataset(X=features, y=targets, ids=dataset['smiles'].values)

# Function to split the dataset
def split_dataset(splitter):
    return splitter.train_test_split(dc_dataset, frac_train=0.8)

# Split using RandomSplitter
random_train, random_test = split_dataset(dc.splits.RandomSplitter())

# Split using ScaffoldSplitter
# Ensure to use SMILES strings for scaffolding
scaffold_splitter = dc.splits.ScaffoldSplitter()
scaffold_train, scaffold_test = scaffold_splitter.train_test_split(dc_dataset, frac_train=0.8, smiles_column='ids')

# Split using MolecularWeightSplitter
mw_train, mw_test = split_dataset(dc.splits.MolecularWeightSplitter())

# Now you have train and test sets for each type of splitter

from dgllife.model.model_zoo.mlp_predictor import MLPPredictor

model = MLPPredictor(in_feats=100, hidden_feats=512, n_tasks=1, dropout=0.)
model


MLPPredictor(
  (predict): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=100, out_features=512, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [11]:
import torch
from torch.utils.data import DataLoader, Dataset
from deepchem.metrics import mean_squared_error


class TorchDatasetWrapper(Dataset):
    """A wrapper for the DeepChem NumpyDataset to make it compatible with PyTorch's DataLoader."""
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return torch.Tensor(self.dataset.X[idx]), torch.Tensor(self.dataset.y[idx])

def train_and_evaluate(train_dataset, test_dataset, epochs=30, batch_size=32, learning_rate=1e-3):
    # Wrap the DeepChem datasets
    train_dataset = TorchDatasetWrapper(train_dataset)
    test_dataset = TorchDatasetWrapper(test_dataset)

    # Create DataLoaders for the training and testing datasets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Define the model
    model = MLPPredictor(in_feats=100, hidden_feats=512, n_tasks=1, dropout=0.)

    # Move the model to the appropriate device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define the loss function and optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        total_loss = 0
        for batch in train_loader:
            batch_x, batch_y = batch
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Forward pass
            pred = model(batch_x)

            # Compute loss
            loss = criterion(pred, batch_y)

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")

    # Evaluation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_x = torch.Tensor(test_dataset.dataset.X).to(device)
        test_y = torch.Tensor(test_dataset.dataset.y).to(device)
        y_pred = model(test_x)
        mse = mean_squared_error(test_y.cpu().numpy(), y_pred.cpu().numpy())
    return mse

# Evaluate for each split
mse_random = train_and_evaluate(random_train, random_test)
print("MSE for Random Split:", mse_random)

mse_scaffold = train_and_evaluate(scaffold_train, scaffold_test)
print("MSE for Scaffold Split:", mse_scaffold)

mse_mw = train_and_evaluate(mw_train, mw_test)
print("MSE for Molecular Weight Split:", mse_mw)


Epoch 1/30, Loss: 0.0684589784969354
Epoch 2/30, Loss: 0.007510011674769755
Epoch 3/30, Loss: 0.003144095797324553
Epoch 4/30, Loss: 0.0018610566739447503
Epoch 5/30, Loss: 0.0013457789186975874
Epoch 6/30, Loss: 0.0010474057423943165
Epoch 7/30, Loss: 0.0007836410972924481
Epoch 8/30, Loss: 0.0007538598408808909
Epoch 9/30, Loss: 0.0007126079321356624
Epoch 10/30, Loss: 0.0007513245696567797
Epoch 11/30, Loss: 0.0007857285911865931
Epoch 12/30, Loss: 0.0007285096267185595
Epoch 13/30, Loss: 0.0010221007224962314
Epoch 14/30, Loss: 0.0009995445087995557
Epoch 15/30, Loss: 0.0011071549940554956
Epoch 16/30, Loss: 0.0013534350402464735
Epoch 17/30, Loss: 0.0016115713168844757
Epoch 18/30, Loss: 0.0013659116604165839
Epoch 19/30, Loss: 0.0014163913095087213
Epoch 20/30, Loss: 0.0012725840232243562
Epoch 21/30, Loss: 0.0015117153934192
Epoch 22/30, Loss: 0.0015940574067783366
Epoch 23/30, Loss: 0.0016016951915281382
Epoch 24/30, Loss: 0.0009531235109967557
Epoch 25/30, Loss: 0.001387071049

You can try varying hidden_feats from 64 to 10000. Let the in_feats be constant.

In [12]:
from deepchem.metrics import r2_score

def train_and_evaluate(train_dataset, test_dataset, hidden_feats, epochs=30, batch_size=32, learning_rate=1e-3):
    train_dataset = TorchDatasetWrapper(train_dataset)
    test_dataset = TorchDatasetWrapper(test_dataset)

    # Define the model with specified hidden_feats
    model = MLPPredictor(in_feats=100, hidden_feats=hidden_feats, n_tasks=1, dropout=0.)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            batch_x, batch_y = batch
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            pred = model(batch_x)
            loss = criterion(pred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        test_x = torch.Tensor(test_dataset.dataset.X).to(device)
        test_y = torch.Tensor(test_dataset.dataset.y).to(device)
        y_pred = model(test_x)
        r2 = r2_score(test_y.cpu().numpy(), y_pred.cpu().numpy())
    return r2

# Train and evaluate models with different hidden_feats
hidden_feats_list = [64, 128, 256, 512, 1024]
for hidden_feats in hidden_feats_list:
    r2 = train_and_evaluate(random_train, random_test, hidden_feats)
    print(f"R2 Score with hidden_feats={hidden_feats}: {r2}")



R2 Score with hidden_feats=64: -0.06385591186452566
R2 Score with hidden_feats=128: 0.1971042849978374
R2 Score with hidden_feats=256: -0.804712426867253
R2 Score with hidden_feats=512: -1.7474898050417695
R2 Score with hidden_feats=1024: -3.1500041614723493


You can try fingerprint lengths of 32, 64, 128, 512, 1024, 2048, 4096. Have a fixed value for hidden_feats

In [19]:
import pandas as pd
import deepchem as dc
from deepchem.metrics import r2_score
import torch
from torch.utils.data import DataLoader, Dataset
from dgllife.model.model_zoo.mlp_predictor import MLPPredictor

# Load and sample the dataset
df = pd.read_csv("https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
dataset = df[["smiles", "homo"]].sample(frac=0.05)

class TorchDatasetWrapper(Dataset):
    """A wrapper for the DeepChem NumpyDataset to make it compatible with PyTorch's DataLoader."""
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return torch.Tensor(self.dataset.X[idx]), torch.Tensor(self.dataset.y[idx])

def featurize_dataset(smiles, fp_size):
    featurizer = dc.feat.CircularFingerprint(size=fp_size, radius=2)
    features = [list(featurizer.featurize(s)[0]) for s in smiles]
    return features

def train_and_evaluate(train_dataset, test_dataset, hidden_feats=128, epochs=30, batch_size=32, learning_rate=1e-3):
    train_dataset = TorchDatasetWrapper(train_dataset)
    test_dataset = TorchDatasetWrapper(test_dataset)

    model = MLPPredictor(in_feats=train_dataset.dataset.X.shape[1], hidden_feats=128, n_tasks=1, dropout=0.)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            batch_x, batch_y = batch
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            pred = model(batch_x)
            loss = criterion(pred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        test_x = torch.Tensor(test_dataset.dataset.X).to(device)
        test_y = torch.Tensor(test_dataset.dataset.y).to(device)
        y_pred = model(test_x)
        r2 = r2_score(test_y.cpu().numpy(), y_pred.cpu().numpy())
    return r2

# Fingerprint lengths to test
fp_lengths = [32, 64, 128, 512, 1024, 2048, 4096]

for fp_length in fp_lengths:
    # Featurize dataset
    features = featurize_dataset(dataset['smiles'], fp_length)
    targets = dataset['homo'].values.reshape(-1, 1)

    # Create DeepChem dataset
    dc_dataset = dc.data.NumpyDataset(X=features, y=targets)

    # Split the dataset
    splitter = dc.splits.RandomSplitter()
    train_set, test_set = splitter.train_test_split(dc_dataset, frac_train=0.8)

    # Train and evaluate
    r2 = train_and_evaluate(train_set, test_set, hidden_feats=128)
    print(f"R2 Score with Fingerprint Length {fp_length}: {r2}")


R2 Score with Fingerprint Length 32: -0.3224411234652602
R2 Score with Fingerprint Length 64: -1.0378350269848995
R2 Score with Fingerprint Length 128: 0.035855875757529465
R2 Score with Fingerprint Length 512: 0.24514210326244879
R2 Score with Fingerprint Length 1024: 0.41569342695917
R2 Score with Fingerprint Length 2048: 0.46243432018985586
R2 Score with Fingerprint Length 4096: 0.5413307773015797
